# Catalog-Based UDTF Registration and Views

This notebook demonstrates how to register UDTFs and Views in Unity Catalog for production use.

## Prerequisites

- **Databricks Runtime 18.1+ (REQUIRED)** - `register_udtfs_and_views()` requires DBR 18.1 or later
- Unity Catalog access with CREATE_CATALOG and CREATE_SCHEMA permissions
- Secret Manager access
- CDF credentials in `config.toml` file
- CDF Data Model with Views

**Note**: For pre-DBR 18.1 environments, use `register_session_scoped_udtfs()` instead (see session-scoped examples).


## Step 1: Install Dependencies


In [ ]:
# Install required packages
%pip install cognite-sdk cognite-databricks databricks-sdk


## Step 2: Load Clients and Setup


In [ ]:
from cognite.pygen import load_cognite_client_from_toml
from cognite.client.data_classes.data_modeling.ids import DataModelId
from cognite.databricks import generate_udtf_notebook, SecretManagerHelper
from databricks.sdk import WorkspaceClient
import tomli  # or tomllib in Python 3.11+

# Load CDF client from TOML
client = load_cognite_client_from_toml("config.toml")

# Create WorkspaceClient (auto-detects credentials in Databricks)
workspace_client = WorkspaceClient()

print("✓ Clients loaded")


In [ ]:
# Cell 2: Register Views (with pre-test validation)
# This cell will verify all UDTFs exist before creating views

view_result = generator.register_views(
    secret_scope=secret_scope,
    if_exists="skip",
    debug=False,
)

print(f"\n✓ Registered {view_result.total_count} View(s)")

# Print registered views
for view in view_result.registered_views:
    if view.view_registered:
        print(f"  ✓ {view.view_name}")
    else:
        print(f"  ✗ {view.view_id}: {view.error_message}")

## Step 3: Setup Secret Manager


In [ ]:
# Read credentials from TOML
with open("config.toml", "rb") as f:
    config = tomli.load(f)

cognite_config = config["cognite"]

# Create Secret Manager helper
secret_helper = SecretManagerHelper(workspace_client)

# Create secret scope (auto-generated name: cdf_{space}_{external_id})
secret_scope = "cdf_sailboat_sailboat"
secret_helper.create_scope_if_not_exists(secret_scope)

# Store credentials in Secret Manager
secret_helper.set_cdf_credentials(
    scope_name=secret_scope,
    project=cognite_config["project"],
    cdf_cluster=cognite_config["cdf_cluster"],
    client_id=cognite_config["client_id"],
    client_secret=cognite_config["client_secret"],
    tenant_id=cognite_config["tenant_id"],
)

print(f"✓ Credentials stored in scope: {secret_scope}")


## Step 4: Generate and Register UDTFs and Views


In [ ]:
# Define data model
data_model_id = DataModelId(
    space="sailboat",
    external_id="sailboat",
    version="1"
)

# Generate UDTFs
generator = generate_udtf_notebook(
    data_model_id,
    client,
    workspace_client=workspace_client,
    catalog="main",  # Unity Catalog catalog name
    schema=None,  # Auto-generated: "sailboat_sailboat_1"
    output_dir="/Workspace/Users/user@example.com/udtf",
)

print("✓ UDTFs generated")


In [ ]:
# Register UDTFs and Views in Unity Catalog
result = generator.register_udtfs_and_views(
    secret_scope=secret_scope,  # Use the scope we created
    if_exists="skip",  # Skip if already exists
    debug=False,
)

print(f"\n✓ Registered {result.total_count} UDTF(s) and View(s)")
print(f"  Catalog: {result.catalog}")
print(f"  Schema: {result.schema_name}")

# Print registered UDTFs and Views
for view_result in result.registered_udtfs:
    print(f"\n  View: {view_result.view_id}")
    print(f"    UDTF: {view_result.udtf_name}")
    print(f"    View: {view_result.view_name}")


## Step 5: Verify in Unity Catalog

You can verify the registration in the Databricks UI:

1. Navigate to **Catalog Explorer**
2. Go to `main > sailboat_sailboat_1`
3. You should see both UDTFs (functions) and Views listed
4. Views are searchable in the Databricks search box


## Step 6: Programmatically Verify Registration


In [ ]:
# Programmatically verify registration
from databricks.sdk import WorkspaceClient

workspace_client = WorkspaceClient()

# List functions in the schema
functions = list(workspace_client.functions.list(
    catalog_name="main",
    schema_name="sailboat_sailboat_1"
))

print(f"\n✓ Found {len(functions)} UDTF(s) in Unity Catalog:")
for func in functions:
    print(f"  - {func.name}")

# List views in the schema
tables = list(workspace_client.tables.list(
    catalog_name="main",
    schema_name="sailboat_sailboat_1"
))

views = [t for t in tables if t.table_type == "VIEW"]
print(f"\n✓ Found {len(views)} View(s) in Unity Catalog:")
for view in views:
    print(f"  - {view.name}")
